# Building Knowledge Graph 

We'll be using the 211 Human Services Taxonomy and the corresponding Taxonomy codes from the IN 211 data to construct an initial KG of the data.  
Afterwards, we'll look for relations and connections between services

In [8]:
import xml.etree.ElementTree as ET 
import xmlschema
from os import path
import networkx as nx
import matplotlib.pyplot as plt
from sys import exit
import json

In [13]:
taxonomy_path = path.abspath('./data/currentTaxonomy_211.xml')
taxo_schema_path = path.abspath('./data/211_taxonomy_xml_schema.xsd')

# validate that taxonomy follows their own schema

if not path.isfile(taxo_schema_path):
    print("Fatal error! Taxonomy schema file {} does not exist".format(taxo_schema_path))
    exit(1)

elif not path.isfile(taxonomy_path):
    print("Fatal error! Taxonomy file {} does not exist".format(taxonomy_path))
    exit(1)
else:

    # describes taxonomy's structure, attributes, etc
    taxo_schema = xmlschema.XMLSchema(taxo_schema_path, converter=xmlschema.AbderaConverter)

    if taxo_schema.is_valid(taxonomy_path):
        
        # load the 211 Human Services taxonomy xml and as a json
        services_taxonomy = ET.parse(taxonomy_path)
        taxonomy_dict = taxo_schema.to_dict(taxonomy_path)
        taxo_root = services_taxonomy.getroot()
        
        taxo_json = json.dumps(taxonomy_dict)
        # dict is very dense and wouldn't print without increasing print IO limit. 
        # save to file for analysis and choosing attrs to keep
        with open('./data/temp_taxo.json', 'w') as tmp_json:
            tmp_json.write(taxo_json)
            
    else: 
        print("Error! The taxonomy doesn't follow provided schema and is invalid. List of errors:")
        for i, err in enumerate(schema_errs):
            print("{}. {}".format(i, err))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
print(taxonomy_dict.keys())

dict_keys(['attributes', 'children'])


In [36]:
# extract info to graph out taxonomy 

%matplotlib inline

def refactorTaxonomy(parent, children, curr_node_num):
    if ('record' not in children and 'children' not in children):
        
        return children
    
    else:
        for child in children:
            curr_node_num += 1
            child_info = child['children']
            child_node = {
                'node_num': curr_node_num,
                'name': child_info['name'],
                'description': child_info['definition'],
                'facet': child_info['facet'],
                'comments': child_info['comments'],
                'keywords': child_info['useReference'],
                'related_concepts': {
                    'code': child_info['relatedConcept']
            }

            refactorTaxonomy(child_node, child_node['children'])
            child_node['children'] = 

            }

            parent['child_nodes'].append()

taxo_root_name = taxonomy_dict['attributes']['name']
taxo_root_node = {'name': taxo_root_name, 'child_nodes': [], 'node_num': 0}
general_categories = taxonomy_dict['children'][0]['record']

In [52]:
for c in general_categories:
    print(c['children'][0]['record'][0]['children'][0]['record'])

Food
Consumer Assistance and Protection
Courts
Educational Institutions/Schools
Environmental Protection and Improvement
Emergency Medical Care
Employment
Death Certification/Burial Arrangements
Counseling Approaches
Arts and Culture
Agencies/Organizations as Recipients
